In [40]:
import json
from datetime import datetime

In [285]:
file_path: str = r'/Users/gonzo/Desktop/RailScope/national_rail_project/src/sandbox/stomp/rtppm/rtppm_data.json'

In [286]:
with open(file_path) as f:
    data = json.load(f)

In [48]:
timestamp: int = int(data['RTPPMDataMsgV1']['timestamp'])

## National performance

- Total:
The total number of trains considered in this performance report.

- OnTime:
The number of trains that arrived on time into their destinations.

- Late:
The number of trains that arrived late to their destinations.

- CancelVeryLate:
The number of trains that were cancelled or arrived very late.

- PPM: <br>
    The Public Performance Measure (PPM) is a measure of train punctuality that combines OnTime and Late trains.<br>
    This dictionary contains the following keys:<br>
    - text: The PPM percentage performance in string format.
    - rag: The Rolling PPM performance category, one of the following (G, A, R, W)
    - ragDisplayFlag: A flag to indicate if the performance category should be displayed or not.
<br>
<br>
- RollingPPM: <br>
    The Rolling Public Performance Measure (Rolling PPM) is a moving average of PPM over a certain period of time. <br>
    This dictionary contains the following keys:
    - text: The Rolling PPM percentage performance in string format.
    - rag: The Rolling PPM performance category, one of the following (G, A, R, W)
    - trendInd: A trend indicator, one of the following (+, = , -)
<br>

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |


In [54]:
def flatten_national_page(nested_dict: dict, parent_key='', sep='_') -> dict:
    """ Recursively flattens a nested dictionary 
    with the following structure:
    
     example: dict =    "NationalPPM": {
                            "Total": "16652",
                            "OnTime": "15235",
                            "Late": "1417",
                            "CancelVeryLate": "499",
                            "PPM": {
                                "text": "91",
                                "rag": "A",
                                "ragDisplayFlag": "Y"
                            },
                            "RollingPPM": {
                                "text": "85",
                                "rag": "R",
                                "trendInd": "-"
                            }
                        },
    """
    items: list = []
    for key, value in nested_dict.items():
        new_key: str = f'{parent_key}{sep}{key}' if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_national_page(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)

In [55]:
national_performance = data['RTPPMDataMsgV1']['RTPPMData']['NationalPage']['NationalPPM']
flatten_national_performance = flatten_national_page(national_performance)
print(flatten_national_performance)

{'Total': '18661', 'OnTime': '17010', 'Late': '1651', 'CancelVeryLate': '566', 'PPM_text': '91', 'PPM_rag': 'A', 'PPM_ragDisplayFlag': 'Y', 'RollingPPM_text': '88', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'}


## National sectors performance
<br>
A dictionary with keys representing different sectors. Each sector is a dictionary containing the following keys:
<br>
<br>

- sectorName: A string representing the name of the sector. <br>

- Total: The total number of trains in the sector <br>

- OnTime: The number of trains that arrived on time to their destinations. <br> 

- Late: The number of trains that arrived late to their destinations. <br> 

- CancelVeryLate: The number of trains that were cancelled or arrived very late. <br> 

- PPM_text: The Public Performance Measure (PPM) percentage performance in string format. <br> 

- PPM_rag: The PPM performance category, one of G (Good performance), A (Medium performance), R (Bad performance), or W (Unknown). <br> 

- RollingPPM_text: The Rolling Public Performance Measure (Rolling PPM) percentage performance in string format. <br> 

- RollingPPM_rag: The Rolling PPM performance category, one of the following (G, A, R, W) <br> 

- RollingPPM_trendInd: A trend indicator, one of the following (+, = , -) <br>

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |


In [56]:
def flatten_national_sector(nested_dict: list, parent_key='', sep='_') -> dict:
    """Iterates over a list of nested dictionaries and flattens them into a dict of dicts 
    
    example: dict =  "Sector": [
                            {
                                "SectorPPM": {
                                    "Total": "8880",
                                    "OnTime": "8202",
                                    "Late": "678",
                                    "CancelVeryLate": "228",
                                    "PPM": {
                                        "text": "92",
                                        "rag": "G"
                                    },
                                    "RollingPPM": {
                                        "text": "84",
                                        "rag": "R",
                                        "trendInd": "-"
                                    }
                                },
                                "sectorCode": "LSE",
                                "sectorDesc": "London and South East"
                            }]
    """
    new_data: dict = {}
    # Traverse over the dicts inside the list
    for sector in nested_dict:
        # Create dict's from sector code and insert name
        new_data[sector["sectorCode"]] = {"sectorName": sector["sectorDesc"]} 
        # Traverse over the nested dict's inside SectorPPM
        for key, value in sector["SectorPPM"].items(): 
            # Explode the nested dicts and add their keys as part of the new key names
            if key == "PPM":
                new_data[sector["sectorCode"]]["PPM_text"] = value["text"]
                new_data[sector["sectorCode"]]["PPM_rag"] = value["rag"]
            elif key == "RollingPPM":
                new_data[sector["sectorCode"]]["RollingPPM_text"] = value["text"]
                new_data[sector["sectorCode"]]["RollingPPM_rag"] = value["rag"]
                new_data[sector["sectorCode"]]["RollingPPM_trendInd"] = value["trendInd"]
            else:
                # Insert data that isn't nested
                new_data[sector["sectorCode"]][key] = value
    return new_data

In [69]:
national_sectors_page = data['RTPPMDataMsgV1']['RTPPMData']['NationalPage']['Sector']
flatten_national_sectors_performance = flatten_national_sector(national_sectors_page)
print(flatten_national_sectors_performance)

{'LSE': {'sectorName': 'London and South East', 'Total': '9973', 'OnTime': '9170', 'Late': '803', 'CancelVeryLate': '270', 'PPM_text': '91', 'PPM_rag': 'A', 'RollingPPM_text': '88', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'}, 'LD': {'sectorName': 'Long Distance', 'Total': '1331', 'OnTime': '1194', 'Late': '137', 'CancelVeryLate': '70', 'PPM_text': '89', 'PPM_rag': 'A', 'RollingPPM_text': '90', 'RollingPPM_rag': 'A', 'RollingPPM_trendInd': '+'}, 'REG': {'sectorName': 'Regional', 'Total': '5393', 'OnTime': '4796', 'Late': '597', 'CancelVeryLate': '198', 'PPM_text': '88', 'PPM_rag': 'R', 'RollingPPM_text': '87', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'}, 'SCO': {'sectorName': 'Scotland', 'Total': '1964', 'OnTime': '1850', 'Late': '114', 'CancelVeryLate': '28', 'PPM_text': '94', 'PPM_rag': 'G', 'RollingPPM_text': '86', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'}}


## National Operator performance

- Total: The total number of trains in the sector. <br>
- PPM: A dictionary with keys 'text' and 'rag', representing the Public Performance Measure percentage and performance category, respectively. <br>
- RollingPPM: A dictionary with keys 'text', 'rag', 'displayFlag', and 'trendInd', representing the Rolling Public Performance Measure percentage, performance category, display flag, and trend indicator, respectively. <br>
- code: A code representing the sector. <br>
- name: The name of the sector. <br>
- keySymbol: An optional key symbol for the sector. <br>

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |

In [170]:
def flatten_operators(nested_dict: list) -> dict:
    """Iterates over a list of nested dictionaries and flattens them into a dict of dicts.
    example: dict = "Operator": [
                        {
                            "Total": "6",
                            "PPM": {
                                "text": "100",
                                "rag": "G"
                            },
                            "RollingPPM": {
                                "text": "-1",
                                "rag": "W",
                                "displayFlag": "Y"
                            },
                            "code": "35",
                            "name": "Caledonian Sleeper",
                            "keySymbol": "*"
                        }]
    """
    new_data: dict = {}
    for record in nested_dict:
        # Create a dictionary to store the flattened record
        flat_record = {}
        # Add all key-value pairs to flat_record
        for key, value in record.items():
            # Add PPM_text and PPM_rag keys for PPM dictionary
            if key == 'PPM':
                flat_record['PPM_text'] = value['text']
                flat_record['PPM_rag'] = value['rag']
            # Add RollingPPM_text, RollingPPM_rag, and RollingPPM_trendInd keys for RollingPPM dictionary
            elif key == 'RollingPPM':
                flat_record['RollingPPM_text'] = value['text']
                flat_record['RollingPPM_rag'] = value['rag']
                flat_record['RollingPPM_trendInd'] = value.get('trendInd')
            elif key != 'code' and key != 'keySymbol':
                # Add the key-value pair directly to flat_record
                flat_record[key] = value
        # Add the flattened record to new_data using code as the key
        new_data[record['code']] = flat_record

    return new_data


In [173]:
national_operator = data['RTPPMDataMsgV1']['RTPPMData']['NationalPage']['Operator']
flatten_national_operator_performance = flatten_operators(national_operator)
print(flatten_national_operator_performance)

{'35': {'Total': '6', 'PPM_text': '100', 'PPM_rag': 'G', 'RollingPPM_text': '-1', 'RollingPPM_rag': 'W', 'RollingPPM_trendInd': None, 'name': 'Caledonian Sleeper'}, '74': {'Total': '310', 'PPM_text': '99', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'Chiltern'}, '30': {'Total': '1413', 'PPM_text': '97', 'PPM_rag': 'G', 'RollingPPM_text': '97', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '=', 'name': 'London Overground'}, '33': {'Total': '593', 'PPM_text': '96', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'Elizabeth line'}, '79': {'Total': '292', 'PPM_text': '96', 'PPM_rag': 'G', 'RollingPPM_text': '92', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '-', 'name': 'c2c'}, '61': {'Total': '147', 'PPM_text': '95', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'London North Eastern Railway'}, '28': {'Total': '434', 'PPM_tex

## OOCPage (Out of Course Page )

- Total: The total number of trains in the sector. <br>
- PPM: A dictionary with keys 'text' and 'rag', representing the Public Performance Measure percentage and performance category, respectively. <br>
- RollingPPM: A dictionary with keys 'text', 'rag', 'displayFlag', and 'trendInd', representing the Rolling Public Performance Measure percentage, performance category, display flag, and trend indicator, respectively. <br>
- code: A code representing the sector. <br>
- name: The name of the sector. <br>
- keySymbol: An optional key symbol for the sector. <br>

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |

In [77]:
occ_page = data['RTPPMDataMsgV1']['RTPPMData']['OOCPage']['Operator']
flatten_occ_page_performance = flatten_operators(occ_page)

In [78]:
print(flatten_occ_page_performance)

{'86': {'Total': '137', 'PPM_text': '97', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'Heathrow Express'}, '55': {'Total': '13', 'PPM_text': '92', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'Hull Trains'}, '45': {'Total': '8', 'PPM_text': '87', 'PPM_rag': 'R', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'Lumo'}, '22': {'Total': '16', 'PPM_text': '81', 'PPM_rag': 'R', 'RollingPPM_text': '-1', 'RollingPPM_rag': 'W', 'RollingPPM_trendInd': None, 'name': 'Grand Central'}}


## FOCPage (Freight Operating Company Page) 

- Total: The total number of trains in the sector. <br>
- PPM: A dictionary with keys 'text' and 'rag', representing the Public Performance Measure percentage and performance category, respectively. <br>
- RollingPPM: A dictionary with keys 'text', 'rag', 'displayFlag', and 'trendInd', representing the Rolling Public Performance Measure percentage, performance category, display flag, and trend indicator, respectively. <br>
- code: A code representing the sector. <br>
- name: The name of the sector. <br>
- keySymbol: An optional key symbol for the sector. <br>

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |

In [82]:
foc_page_national_performance = data['RTPPMDataMsgV1']['RTPPMData']['FOCPage']['NationalPPM']
foc_page_national_operator = data['RTPPMDataMsgV1']['RTPPMData']['FOCPage']['Operator']
flatten_foc_page_national_performance = flatten_national_page(foc_page_national_performance)
flatten_foc_page_national_operator_performance = flatten_operators(foc_page_national_operator)

In [80]:
print(flatten_foc_page_national_performance)

{'Total': '406', 'OnTime': '350', 'Late': '56', 'PPM_text': '86', 'PPM_rag': 'A', 'PPM_ragDisplayFlag': 'Y', 'RollingPPM_text': '81', 'RollingPPM_rag': 'A', 'RollingPPM_trendInd': '-'}


In [83]:
print(flatten_foc_page_national_operator_performance)

{'42': {'Total': '27', 'PPM_text': '92', 'PPM_rag': 'G', 'RollingPPM_text': '66', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-', 'name': 'Colas Freight'}, '05': {'Total': '150', 'PPM_text': '92', 'PPM_rag': 'G', 'RollingPPM_text': '100', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '+', 'name': 'DB Cargo'}, 'FLI': {'Total': '40', 'PPM_text': '90', 'PPM_rag': 'G', 'RollingPPM_text': '87', 'RollingPPM_rag': 'G', 'RollingPPM_trendInd': '-', 'name': 'Freightliner Intermodal'}, '54': {'Total': '92', 'PPM_text': '82', 'PPM_rag': 'A', 'RollingPPM_text': '66', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-', 'name': 'GB Railfreight'}, 'NRM': {'Total': '97', 'PPM_text': '82', 'PPM_rag': 'A', 'RollingPPM_text': '72', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-', 'name': 'Network Rail - Materials '}, 'FHH': {'Total': '77', 'PPM_text': '77', 'PPM_rag': 'R', 'RollingPPM_text': '76', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-', 'name': 'Freightliner Heavy Haul'}, '97': {'Total': '2

## OperatorPage

The OperatorPage provides performance data for individual train operating companies. It contains a list of dictionaries, with each dictionary providing data for a single operator. 

### Operator

The "Operator" dictionary provides performance data for a single operator. The fields in this dictionary include:

- Total: The total number of trains operated by the operator in the period.
- OnTime: The number of trains that arrived on time into their destinations.
- Late: The number of trains that arrived late to their destinations.
- CancelVeryLate: The number of trains that were cancelled or arrived very late.
- PPM: The Public Performance Measure (PPM) percentage performance for the operator.
- RollingPPM: The Rolling Public Performance Measure (Rolling PPM) percentage performance for the operator.
- code: The unique code assigned to the operator.
- name: The name of the operator.

The PPM and RollingPPM fields provide information about the punctuality and reliability of the operator's services. The "rag" field in both PPM and RollingPPM indicates the performance category, with "G" representing Good performance and "R" representing Bad performance. The "displayFlag" field in RollingPPM indicates whether or not the performance category should be displayed, while the "trendInd" field indicates the direction of the trend in RollingPPM performance. 

### OprToleranceTotal

The "OprToleranceTotal" dictionary provides information about the tolerance levels set for the operator. The fields in this dictionary include:

- Total: The total number of trains considered in the tolerance calculation.
- OnTime: The number of trains that were within the tolerance band.
- Late: The number of trains that were outside the tolerance band.
- CancelVeryLate: The number of trains that were cancelled or arrived very late.
- timeband: The tolerance band in minutes.

| rag | Performance category             | trendInd | Trend direction         |
| --- | ---                              | ---      | ---                     |
| G   | Good performance                 | `+`      | Rising trend            |
| A   | Medium performance               | `=`      | No change/flat trend    |
| R   | Bad performance                  | `-`      | Falling trend           |
| W   | Unknown                          | N/A      | N/A                     |

In [346]:
def flatten_operators_page(nested_dicts: list) -> dict:
    new_data = {}

    for operator in nested_dicts:
        # Create a new dictionary to hold the flattened data for this operator
        flat_dict = {}

        # Extract data from the operator dictionary and flatten it
        op_data = operator['Operator']
        flat_dict['sectorName'] = op_data['name']
        flat_dict['total'] = op_data['Total']
        flat_dict['onTime'] = op_data['OnTime']
        flat_dict['late'] = op_data['Late']
        flat_dict['cancelVeryLate'] = op_data['CancelVeryLate']
        flat_dict['PPM_text'] = op_data['PPM']['text']
        flat_dict['PPM_rag'] = op_data['PPM']['rag']
        flat_dict['RollingPPM_text'] = op_data['RollingPPM']['text']
        flat_dict['RollingPPM_rag'] = op_data['RollingPPM']['rag']
        flat_dict['RollingPPM_trendInd'] = op_data['RollingPPM'].get('trendInd')

        # Add the flattened operator data to the new_data dictionary using the operator code as the key
        new_data[op_data['code']] = flat_dict

    return new_data

In [347]:
operator_page = data['RTPPMDataMsgV1']['RTPPMData']['OperatorPage']

In [348]:
flatten_operator_page_performance = flatten_operators_page(operator_page)

In [352]:
for record in flatten_operator_page_performance.items():
    print(record)

('97', {'sectorName': 'Direct Rail Services', 'total': '20', 'onTime': '14', 'late': '6', 'cancelVeryLate': '0', 'PPM_text': '70', 'PPM_rag': 'R', 'RollingPPM_text': '66', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'})
('FHH', {'sectorName': 'Freightliner Heavy Haul', 'total': '77', 'onTime': '60', 'late': '17', 'cancelVeryLate': '0', 'PPM_text': '77', 'PPM_rag': 'R', 'RollingPPM_text': '76', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'})
('27', {'sectorName': 'CrossCountry', 'total': '205', 'onTime': '162', 'late': '43', 'cancelVeryLate': '29', 'PPM_text': '79', 'PPM_rag': 'R', 'RollingPPM_text': '80', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '+'})
('22', {'sectorName': 'Grand Central', 'total': '16', 'onTime': '13', 'late': '3', 'cancelVeryLate': '2', 'PPM_text': '81', 'PPM_rag': 'R', 'RollingPPM_text': '-1', 'RollingPPM_rag': 'W', 'RollingPPM_trendInd': None})
('NRM', {'sectorName': 'Network Rail - Materials ', 'total': '97', 'onTime': '80', 'late': '17', 'cancelVeryL

In [457]:
operator_page_groups = data['RTPPMDataMsgV1']['RTPPMData']['OperatorPage']

def flatten_operators_page_groups(nested_dicts: list) -> list:
    """
    Iterates over a list of nested dicts and retrieves only the service groups if exist
    After that it flattens the nested dicts inside the service groups and
    returns a list with each dict as a record

    example: dict = ["OprServiceGrp": [
                        {
                            "Total": "86",
                            "OnTime": "73",
                            "Late": "13",
                            "CancelVeryLate": "5",
                            "PPM": {
                                "text": "84",
                                "rag": "R"
                            },
                            "RollingPPM": {
                                "text": "82",
                                "rag": "R",
                                "displayFlag": "Y",
                                "trendInd": "-"
                            },
                            "name": "CrossCountry Inter City",
                            "timeband": "10",
                            "sectorCode": "LD"
                        },
                        {
                            "Total": "119",
                            "OnTime": "89",
                            "Late": "30",
                            "CancelVeryLate": "24",
                            "PPM": {
                                "text": "74",
                                "rag": "R"
                            },
                            "RollingPPM": {
                                "text": "77",
                                "rag": "R",
                                "displayFlag": "Y",
                                "trendInd": "+"
                            },
                            "name": "CrossCountry Local & Provincial",
                            "timeband": "10",
                            "sectorCode": "LD"
                        }
                    ]]
    """
    nested_groups: list = []
    service_groups: list = []

    # Iterate over the nested dicts and retrieve only the nested service groups if exist
    for operator_groups in nested_dicts:
        try:
            nested_groups.extend(operator_groups['OprServiceGrp'])
        except KeyError as e:
            pass
    
    # Theres a bug in my code which returns the nested_groups with non-dict elements.
    nested_groups: list = [record for record in nested_groups if isinstance(record, dict)]
    
    #
    for service_group in nested_groups:
        # Create a single record for each nested service group
        record: dict = {}

        try:
            record['name'] = service_group['name']
            record['Total'] = service_group['Total']
            record['OnTime'] = service_group['OnTime']
            record['Late'] = service_group['Late']
            record['CancelVeryLate'] = service_group['CancelVeryLate']
            record['PPM_text'] = service_group.get('PPM', {}).get('text', '')
            record['PPM_rag'] = service_group.get('PPM', {}).get('rag', '')
            record['RollingPPM_text'] = service_group.get('RollingPPM', {}).get('text', '')
            record['RollingPPM_rag'] = service_group.get('RollingPPM', {}).get('rag', '')
            record['RollingPPM_trendInd'] = service_group.get('RollingPPM', {}).get('trendInd', '')
            service_groups.append(record)
        except KeyError as e:
            # logging.info(e)
            pass

    return service_groups

In [459]:
result = flatten_operators_page_groups(operator_page_groups)

for record in result:
    print(record)
    print('-'*100)

{'name': 'CrossCountry Inter City', 'Total': '86', 'OnTime': '73', 'Late': '13', 'CancelVeryLate': '5', 'PPM_text': '84', 'PPM_rag': 'R', 'RollingPPM_text': '82', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '-'}
----------------------------------------------------------------------------------------------------
{'name': 'CrossCountry Local & Provincial', 'Total': '119', 'OnTime': '89', 'Late': '30', 'CancelVeryLate': '24', 'PPM_text': '74', 'PPM_rag': 'R', 'RollingPPM_text': '77', 'RollingPPM_rag': 'R', 'RollingPPM_trendInd': '+'}
----------------------------------------------------------------------------------------------------
{'name': 'Kings X - Bradford', 'Total': '5', 'OnTime': '5', 'Late': '0', 'CancelVeryLate': '0', 'PPM_text': '100', 'PPM_rag': 'G', 'RollingPPM_text': '-1', 'RollingPPM_rag': 'W', 'RollingPPM_trendInd': ''}
----------------------------------------------------------------------------------------------------
{'name': 'Kings X - Sunderland', 'Total': '11', 'OnTi